In [3]:
# import

import json
import statistics
import math

import networkx as nx
import numpy as np
import pandas as pd
import seaborn as sns
import optuna
from scipy.spatial import Delaunay
from scipy.sparse.csgraph import minimum_spanning_tree

%matplotlib inline
import matplotlib.pyplot as plt

from egraph import Graph, Coordinates, Rng, SparseSgd


/Users/fuga_takata/dev/vdslab-project/hyperparameter_optimization/.venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
EDGE_WEIGHT = 30
K_FROM = 1
K_TO = 20

LARGE_DATASET_NAMES = [
    '3elt.json',
    '1138_bus.json',
    'dwt_1005.json',
    'dwt_2680.json',
    'poli.json',
    'qh882.json',
    'USpowerGrid.json',
]
SMALL_DATASET_NAMES = [
    'bull.json',
    'chvatal.json',
    'cubical.json',
    'davis_southern_women.json',
    'desargues.json',
    'diamond.json',
    'dodecahedral.json',
    'florentine_families.json',
    'frucht.json',
    'heawood.json',
    'hoffman_singleton.json',
    'house_x.json',
    'house.json',
    'icosahedral.json',
    'karate_club.json',
    'krackhardt_kite.json',
    'les_miserables.json',
    'moebius_kantor.json',
    'octahedral.json',
    'pappus.json',
    'petersen.json',
    'sedgewick_maze.json',
    'tutte.json',
]

DATASET_NAMES = SMALL_DATASET_NAMES

In [5]:
# グラフの生成・読み込み

def generate_graph_from_nx_graph(nx_graph):
    graph = Graph()

    indices = {}
    for u in nx_graph.nodes:
        indices[u] = graph.add_node(u)
    for u, v in nx_graph.edges:
        graph.add_edge(indices[u], indices[v], (u, v))

    return graph, indices


def graph_preprocessing(nx_graph):
    nx_graph = nx.Graph(nx_graph)

    # グラフを無向グラフに
    nx_graph = nx_graph.to_undirected()

    # エッジのループを除去
    nx_graph.remove_edges_from(list(nx.selfloop_edges(nx_graph)))

    # 最大連結成分を用いる
    largest_cc = max(nx.connected_components(nx_graph), key=len)
    largest_cc_graph = nx_graph.subgraph(largest_cc)

    new_graph = nx.Graph()
    nodes = [str(node_id) for node_id in largest_cc_graph.nodes]
    new_graph.add_nodes_from(nodes)

    # エッジにidと重みを追加
    for i, edge in enumerate(largest_cc_graph.edges):
        new_graph.add_edge(str(edge[0]), str(edge[1]), weight=EDGE_WEIGHT, id=str(i))
        # weighted_edges.append((str(edge[0]), str(edge[1]), EDGE_WEIGHT))

    return new_graph


nx_graphs = []
dir = 'lib/egraph-rs/js/dataset/'
for filename in DATASET_NAMES:
    with open(dir + filename) as f:
        graph_data = json.load(f)

    nx_graph = nx.node_link_graph(graph_data)
    nx_processed_graph = graph_preprocessing(nx_graph)
    nx_graphs.append(nx_processed_graph)

# 追加のnx.scale_free_graph
for i in range(1, 3 + 1):
    n = i * 1000
    nx_graphs.append(graph_preprocessing(nx.scale_free_graph(n)))

    DATASET_NAMES.append(f'nx_scale_free_{n}')

graph_data = []
for nx_graph in nx_graphs:
    graph, indices = generate_graph_from_nx_graph(nx_graph)
    graph_data.append((graph, indices))


In [6]:
# shape graph生成


def generate_k_nearest_graph(pos, K, distances=None):
    k_nearest_graph = nx.Graph()

    nodes = [node_id for node_id in pos]
    k_nearest_graph.add_nodes_from(nodes)

    if distances is None:
        distances = {}
        for source in pos:
            distances[source] = []
            source_pos = np.array(pos[source])
            for target in pos:
                if target == source:
                    continue
                target_pos = np.array(pos[target])
                distance = np.linalg.norm(source_pos - target_pos)

                distances[source].append({'id': target, 'distance': distance})

        for source in pos:
            distances[source] = sorted(
                distances[source], key=lambda x: x['distance'])

    weighted_edges = []
    for source in pos:
        for distance in distances[source][:K]:
            weighted_edges.append((source, distance['id'], EDGE_WEIGHT))
    k_nearest_graph.add_weighted_edges_from(weighted_edges)

    return k_nearest_graph, distances


def generate_degree_based_k_nearest(nx_graph, pos, distances=None, degrees=None):
    k_nearest_graph = nx.Graph()

    nodes = [node_id for node_id in pos]
    k_nearest_graph.add_nodes_from(nodes)

    if distances is None:
        distances = {}
        for source in pos:
            distances[source] = []
            source_pos = np.array(pos[source])
            for target in pos:
                if target == source:
                    continue
                target_pos = np.array(pos[target])
                distance = np.linalg.norm(source_pos - target_pos)

                distances[source].append({'id': target, 'distance': distance})

        for source in pos:
            distances[source] = sorted(
                distances[source], key=lambda x: x['distance'])

    if degrees is None:
        degrees = {}
        for node_id in pos:
            degrees[node_id] = nx_graph.degree[node_id]

    weighted_edges = []
    for source in pos:
        for distance in distances[source][:degrees[node_id]]:
            weighted_edges.append((source, distance['id'], EDGE_WEIGHT))
    k_nearest_graph.add_weighted_edges_from(weighted_edges)

    return k_nearest_graph, distances, degrees


def generate_delaunay_triangulation_graph(pos):
    index_id_map = {}
    pos_array = []
    for index, node_id in enumerate(pos):
        positions = pos[node_id]
        position = [positions[0], positions[1]]
        pos_array.append(position)
        index_id_map[index] = node_id

    pos_ndarray = np.array(pos_array)
    delaunay = Delaunay(pos_ndarray)

    delaunay_triangulation_graph = nx.Graph()

    nodes = [node_id for node_id in pos]
    delaunay_triangulation_graph.add_nodes_from(nodes)

    weighted_edges = []
    for n in delaunay.simplices:
        n0 = n[0]
        n1 = n[1]
        n2 = n[2]
        weighted_edges.append(
            (index_id_map[n0], index_id_map[n1], EDGE_WEIGHT))
        weighted_edges.append(
            (index_id_map[n0], index_id_map[n2], EDGE_WEIGHT))
        weighted_edges.append(
            (index_id_map[n1], index_id_map[n2], EDGE_WEIGHT))
    delaunay_triangulation_graph.add_weighted_edges_from(weighted_edges)

    return delaunay_triangulation_graph


def generate_mst_graph(pos):
    index_id_map = {}
    pos_array = []
    for index, node_id in enumerate(pos):
        positions = pos[node_id]
        position = [positions[0], positions[1]]
        pos_array.append(position)
        index_id_map[index] = node_id

    dists = []
    for source_pos in pos_array:
        source_pos_ndarray = np.array(source_pos)
        tmp = []
        for target_pos in pos_array:
            target_pos_ndarray = np.array(target_pos)
            tmp.append(np.linalg.norm(source_pos_ndarray - target_pos_ndarray))
        dists.append(tmp)

    dists_ndarray = np.array(dists)
    mst_r = minimum_spanning_tree(dists_ndarray)

    new_nx_graph = nx.Graph()
    nodes = [node_id for node_id in pos]
    new_nx_graph.add_nodes_from(nodes)
    weighted_edges = []
    for mst_v in list(zip(*mst_r.nonzero())):
        source_id = index_id_map[mst_v[0]]
        target_id = index_id_map[mst_v[1]]
        weighted_edges.append((source_id, target_id, EDGE_WEIGHT))
    new_nx_graph.add_weighted_edges_from(weighted_edges)

    return new_nx_graph


In [7]:
# 類似度

def jaccard_similarity_sum(nx_graph, nx_shape_graph):
    j_s_sum = 0
    for node in nx_graph.nodes:
        g_n = [n for n in nx_graph.neighbors(node)]
        s_n = [n for n in nx_shape_graph.neighbors(node)]
        and_n = list(set(g_n) & set(s_n))
        or_n = list(set(g_n + s_n))

        j_s_sum += len(and_n) / len(or_n)

    return j_s_sum / len(nx_graph.nodes)


In [8]:
def ideal_edge_length(nx_graph, pos):
    s = 0
    for source, target, data in nx_graph.edges(data=True):
        weight = data['weight'] if 'weight' in data else 1
        shortest_path_length = nx.shortest_path_length(
            nx_graph, source, target, data['weight'] if 'weight' in data else None)

        lij = shortest_path_length * weight
        dist = np.linalg.norm(np.array(pos[source]) - np.array(pos[target]))
        s += ((dist - lij) / lij) ** 2

    return s


In [9]:
def is_edge_crossing(p1, p2, p3, p4):
    tc1 = (p1[0] - p2[0]) * (p3[1] - p1[1]) + (p1[1] - p2[1]) * (p1[0] - p3[0])
    tc2 = (p1[0] - p2[0]) * (p4[1] - p1[1]) + (p1[1] - p2[1]) * (p1[0] - p4[0])
    td1 = (p3[0] - p4[0]) * (p1[1] - p3[1]) + (p3[1] - p4[1]) * (p3[0] - p1[0])
    td2 = (p3[0] - p4[0]) * (p2[1] - p3[1]) + (p3[1] - p4[1]) * (p3[0] - p2[0])
    return tc1 * tc2 < 0 and td1 * td2 < 0


In [10]:
def edge_crossing_finder(nx_graph, pos):
    edges = {}
    for s1, t1, attr1 in nx_graph.edges(data=True):
        id1 = attr1['id']
        if id1 not in edges:
            edges[id1] = {}
        for s2, t2, attr2 in nx_graph.edges(data=True):
            id2 = attr2['id']
            crossing = is_edge_crossing(pos[s1], pos[t1], pos[s2], pos[t2])
            edges[id1][id2] = crossing

    return edges


In [11]:
def crossing_number(nx_graph, pos, edge_crossing=None):
    if edge_crossing is None:
        edge_crossing = edge_crossing_finder(nx_graph, pos)

    s = 0
    for i in edge_crossing:
        if edge_crossing[i]:
            s += 1

    return s

In [12]:
def crossing_angle_maximization(nx_graph, pos, edge_crossing=None):
    if edge_crossing is None:
        edge_crossing = edge_crossing_finder(nx_graph, pos)
    s = 0
    for s1, t1, attr1 in nx_graph.edges(data=True):
        id1 = attr1['id']
        for s2, t2, attr2 in nx_graph.edges(data=True):
            id2 = attr2['id']
            if edge_crossing[id1][id2] or edge_crossing[id2][id1]:
                e1 = np.array(pos[s1]) - np.array(pos[t1])
                e2 = np.array(pos[s2]) - np.array(pos[t2])
                s += np.dot(e1, e2) / (np.linalg.norm(e1) * np.linalg.norm(e2))

    return s


In [13]:
def gravity_center(pos):
    gx = 0
    gy = 0
    for node_id in pos:
        x, y = pos[node_id]
        gx += x
        gy += y

    gx /= len(pos)
    gy /= len(pos)

    return gx, gy


def aspect_ratio(pos):
    gravity_centered_pos = []
    gx, gy = gravity_center(pos)
    for node_id in pos:
        x, y = pos[node_id]
        gravity_centered_pos.append([x - gx, y - gy])
    gravity_centered_pos = np.array(gravity_centered_pos)
    _, s, _ = np.linalg.svd(gravity_centered_pos, full_matrices=True)
    a = max(s)
    b = min(s)

    return b / a


In [29]:
def get_max_degree(nx_graph):
    md = 0
    for node_id in nx_graph.nodes:
        d = nx_graph.degree[node_id]
        if md < d:
            md = d

    return md

# すべてのノードについて、あるノードに入射するエッジ同士のなす角度が最も小さいもの
def angular_resolution(nx_graph, pos):
    edges = {}
    for s, t in nx_graph.edges:
        if s not in edges:
            edges[s] = {}
        if t not in edges:
            edges[t] = {}
        edges[s][t] = True
        edges[t][s] = True

    ls = 0
    for s in pos:
        if s not in edges:
            continue
        l = 2 * math.pi
        ts = sorted([node_id for node_id in edges[s]])
        for i, t1 in enumerate(ts):
            e1 = np.array(pos[s]) - np.array(pos[t1])
            for t2 in ts[i + 1:]:
                if t1 == t2 or s == t1 or s == t2:
                    continue
                e2 = np.array(pos[s]) - np.array(pos[t2])
                angle = math.acos(
                    np.dot(e1, e2) / (np.linalg.norm(e1) * np.linalg.norm(e2)))
                if angle < l:
                    l = angle
        ls += l
    return ls


In [32]:
# ノードのユークリッド距離として定義される。stressなどと一致させるために正規化するらしいけど多分必要ない
def node_resolution(pos):
    nodes = sorted([node_id for node_id in pos])
    r = 0
    for i, sid in enumerate(nodes):
        for tid in nodes[i + 1:]:
            dist = np.linalg.norm(np.array(pos[sid]) - np.array(pos[tid]))
            if dist < r:
                r = dist

    return r


In [42]:
# エッジを直径とした円の内部にノードを含まないようにしたい。
# そこで内部に含まれる点
def gabriel_graph_property(nx_graph,pos):
    s = 0
    for e in nx_graph.edges:
        e1, e2 = e
        x1 = np.array(pos[e1])
        x2 = np.array(pos[e2])
        rij = np.linalg.norm(x1 - x2) / 2
        cij = (np.array(x1) + np.array(x2)) / 2
        for node_id in nx_graph.nodes:
            if e1 == node_id or e2 == node_id:
                continue
            s += max(0, rij - np.linalg.norm(np.array(pos[node_id] - cij)))

    return s

In [19]:
# グラフの描画

def draw_graph(graph, indices):
    drawing = Coordinates.initial_placement(graph)
    rng = Rng.seed_from(0)  # random seed
    sgd = SparseSgd(
        graph,
        lambda _: 30,  # edge length
        50,  # number of pivots
        rng,
    )
    scheduler = sgd.scheduler(
        100,  # number of iterations
        0.1,  # eps: eta_min = eps * min d[i, j] ^ 2
    )

    def step(eta):
        sgd.shuffle(rng)
        sgd.apply(drawing, eta)

    scheduler.run(step)

    pos = {u: (drawing.x(i), drawing.y(i)) for u, i in indices.items()}

    return pos


In [45]:
shape_based_metrics_values = {}
positions = {}

for nx_graph, data, filename in zip(nx_graphs, graph_data, DATASET_NAMES):
    graph, indices = data
    pos = draw_graph(graph, indices)

    graph_degrees = None
    distances = None

    shape_based_metrics_values[filename] = {}
    positions[filename] = pos

    # shape graph by delaunay triangulation
    nx_shape_graph = generate_delaunay_triangulation_graph(pos)
    shape_based_metrics_values[filename][f'd-t'] = jaccard_similarity_sum(
        nx_graph, nx_shape_graph)

    break